<a href="https://colab.research.google.com/github/juliawol/WB_Knowledge_Base/blob/main/Deploy_to_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import gradio as gr
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
import torch
import re


model = SentenceTransformer('/content/fine_tuned_model_with_triplets')


chunks_data_path = '/content/chunks.csv'
chunks_df = pd.read_csv(chunks_data_path)
original_chunks = chunks_df['Chunk'].tolist()


chunk_embeddings = model.encode(original_chunks, convert_to_tensor=True)


tokenizer = AutoTokenizer.from_pretrained('DiTy/cross-encoder-russian-msmarco')
cross_encoder_model = AutoModelForSequenceClassification.from_pretrained('DiTy/cross-encoder-russian-msmarco')


def embed_texts(texts):
    return model.encode(texts, convert_to_tensor=True)

def find_relevant_chunks(question_embedding, top_k=5):
    cosine_similarities = cosine_similarity(question_embedding.cpu().numpy(), chunk_embeddings.cpu().numpy()).flatten()
    num_candidates = top_k * 10  # Adjust to get more candidates for re-ranking
    top_indices = cosine_similarities.argsort()[-num_candidates:][::-1]
    return [original_chunks[i] for i in top_indices]


def re_rank(question, candidate_chunks):
    inputs = tokenizer([question] * len(candidate_chunks), candidate_chunks, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        scores = cross_encoder_model(**inputs).logits.squeeze()
    ranked_indices = scores.argsort(descending=True)
    return [candidate_chunks[i] for i in ranked_indices]


def find_relevant_chunks_with_reranking(question, top_k=5):
    question_embedding = embed_texts([question])
    candidate_chunks = find_relevant_chunks(question_embedding, top_k=top_k)
    ranked_chunks = re_rank(question, candidate_chunks) if len(candidate_chunks) > 1 else candidate_chunks
    return ranked_chunks[:top_k]


def answer_question(question):

    if not question or len(question) < 10:
        return "Пожалуйста, задайте вопрос. Количество символов должно превышать 10."


    if not re.search(r'[а-яА-Я]', question):
        return "Простите, на этом языке я пока не говорю. Попробуем еще раз?"


    top_chunks = find_relevant_chunks_with_reranking(question, top_k=5)


    if not top_chunks:
        return "Ничего не нашлось. Я только учусь, сформулируйте вопрос иначе, пожалуйста"


    return "\n\n".join([f"Answer {i+1}: {chunk}" for i, chunk in enumerate(top_chunks)])

# Set up Gradio interface
iface = gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs="text",
    title="Question Answering Model",
    description="Здравствуйте! Задайте мне вопрос на русском о работе пунктов выдачи WB, и я постараюсь найти самые лучшие ответы."
)

# Launch the Gradio interface with shareable link
iface.launch(share=True)


URL: https://54bac118d6e06c6b04.gradio.live

Здравствуйте! Эта ссылка будет доступна в течение всего трех дней. Если нам потребуется больше времени, пожалуйста, сообщите! Я запущу приложение заново. Мой телеграм для связи @juliawolkenstein
